# Potential Outcome


We will use the [potential outcome framework](https://en.wikipedia.org/wiki/Rubin_causal_model) in this note. There are many other approaches including the [direct acyclic graphs](https://en.wikipedia.org/wiki/Causal_graph).

Suppose we **observe** a data set with treatment $Z_i$ ($A$ or $B$) and outcome $Y_i$ (1 for success and 0 for fail) for $i=1,2,\ldots, n$. Further consider the **potential outcomes** $Y_i( \text{treatment} = A)$ that represents the outcome if subject $i$ receives treatment $A$, and $Y_i( \text{treatment} = B)$,  the outcome if subject $i$ receives treatment $B$. Using the notation of potential outcomes, we can write the observed outcome as $Y_i(Z_i)$, which shows that only one potential outcome is observed for each unit. The following table show the potential outcomes and the **observed values** in orange. 


In [2]:

import numpy as np, ipywidgets as wd
from scipy.stats import ttest_ind
from IPython.display import display, clear_output

# ------------------------------------------------------------------ data ----
rng      = np.random.default_rng(2)
n_subj   = 10
Y0 = rng.normal(70, 10, n_subj)                 # potential control outcomes
Y1 = Y0 + rng.normal(0, 3,  n_subj)             # potential treatment outcomes

# ----- containers -----------------------------------------------------------
selected  = [None]*n_subj       # None / 0 / 1
observed  = [None]*n_subj

out = wd.Output()

def reset_state():
    """Clear selections & visuals."""
    for c in range(n_subj):
        selected[c] = None
        observed[c] = None
        for r in range(2):
            btn = buttons[r][c]
            btn.disabled = False
            btn.style.button_color = None
    with out: clear_output()

# ----------------- statistic & display --------------------------------------
def show_results():
    treat   = [v for s,v in zip(selected, observed) if s == 1]
    control = [v for s,v in zip(selected, observed) if s == 0]
    diff    = np.mean(control) - np.mean(treat)
    t_stat, p_val = ttest_ind(control, treat, equal_var=False)
    with out:
        clear_output()
        print("----- Results -----")
        print(f"Control mean = {np.mean(control):.2f}")
        print(f"Treat   mean = {np.mean(treat):.2f}")
        print(f"Mean diff (C − T) = {diff:.2f}")
        print(f"Welch t = {t_stat:.3f}")
        print(f"p-value = {p_val:.3f}")

# ----------------- button factory -------------------------------------------
def make_button(row, col):
    val = Y0[col] if row == 0 else Y1[col]
    b = wd.Button(description=f"{val:.1f}",
                  layout=wd.Layout(width="65px"),
                  tooltip=f"Subj {col+1} | Y{row}")
    def on_click(_):
        if selected[col] is not None: return
        selected[col] = row
        observed[col] = val
        # visual updates
        for r in range(2):
            buttons[r][col].disabled = True
        b.style.button_color = "orange"
        if None not in selected:
            show_results()
    b.on_click(on_click)
    return b

# ----------------- build grid -----------------------------------------------
buttons = [[make_button(r,c) for c in range(n_subj)] for r in range(2)]
grid = wd.VBox([wd.HBox(buttons[0]), wd.HBox(buttons[1])])

# ----------------- randomize logic ------------------------------------------
def randomize(_):
    reset_state()
    idx = rng.permutation(n_subj)
    ctrl_cols, treat_cols = idx[:5], idx[5:]
    # programmatic "click"
    for c in ctrl_cols:
        buttons[0][c].click()
    for c in treat_cols:
        buttons[1][c].click()

rand_btn  = wd.Button(description="Randomize assignment",
                      button_style='success')
rand_btn.on_click(randomize)

reset_btn = wd.Button(description="Reset", button_style='warning')
reset_btn.on_click(lambda _: reset_state())

# ----------------- assemble UI ----------------------------------------------
header = wd.HTML(
    "<b>Potential-outcome playground</b><br>"
    "Choose <i>one</i> cell per column.<br>"
    "<span style='color:blue'>Top</span> = control Y₀,&nbsp;"
    "<span style='color:blue'>bottom</span> = treatment Y₁."
)

ui = wd.VBox([header, grid, wd.HBox([rand_btn, reset_btn]), out])
display(ui)



With these new notations, we can see that the what-ifs describe the comparison of potential outcomes, e.g., $Y_i(A)-Y_i(B)$. To be specific, a causal effect is  defined to be the comparison of the  potential outcomes on the **same units**.
- **Individual causal effect**:  $Y_i(A)-Y_i(B)$.
- **Average causal effect** (ACE): $\text{mean}\{Y_i(A)-Y_i(B)\}$

It is important to notice that $\text{mean}\{Y_i(A)\}$ does not equal to $\text{mean}\{Y_i \mid Z_i=A\} $ in general. We can see this using the examples in the above tables.
- $\text{mean}\{Y_i(A)\}$: average of $Y_i(A)$ for units 1, 2, 3, 4.
- $\text{mean}\{Y_i\mid Z_i=A\}$: average of $Y_i$ for units 1,3.




The above tables also reveal the fundamental problem of causal inference. That is only one potential outcome is observed.  In randomized experiments, randomization ensures $\text{mean}\{Y_i(A)\} = \text{mean}\{Y_i \mid Z_i=A\} $, which allows us to estimate the average causal effects. We should always think about what can and what cannot be learned from the observed data.

> If your experiment needs statistics, you ought to have done a better experiment.
>
>        --  Ernest Rutherford 
        

> Regression models often seem to be used to compensate for problems in measurement, data collection, and study design. By the time models are deployed, the scientific position is nearly hopeless. 
> 
>        --  David Freedman 



### Identification and Randomization  

A new task that will be often discussed in this chapter is **Identification**: what can be identified if there are infinite amount of data. This tasks concerns the design of experiements or how the data is collected. In contrast, the **Inference** problem, where we study what can be learned given a finite sample, is what we have focused on in most of the statistical methods. 

We can now examine why causal interpretation is possible from randomized experiments. Recall that the average treatment effect is defined as  
$$
{\rm ACE} \equiv \mathbb{E}[Y(1)-Y(0)]=\mathbb{E}[Y(1)]-\mathbb{E}[Y(0)].
$$
**Randomization of treatments means $\{Z_i\}_{i=1}^n \perp \{ Y_i(0), Y_i(1)\}_{i=1}^n$.**

We can derive 
\begin{align}
{\rm ACE} & = \mathbb{E}[Y(1)]-\mathbb{E}[Y(0)]=\mathbb{E}[Y(1)|Z=1]-\mathbb{E}[Y(0)|Z=0]\\
& = \mathbb{E}[Y|Z=1]-\mathbb{E}[Y|Z=0],
\end{align}
where the second equality follows from independence between treatments and potential outcomes. 